In [761]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud
%matplotlib inline
sns.set_style("whitegrid")

#set warning 
import warnings
warnings.filterwarnings('ignore')


pd.pandas.set_option('display.max_columns', None)

In [762]:
filename = "dataset_youtube-comments-scraper_2025-09-17_04-18-13-435.csv"
df = pd.read_csv(filename, encoding = 'latin-1')
df

,"ï»¿""author""",comment,commentsCount,pageUrl,replyCount,title,type,videoID,voteCount
0,@Mahidin.s-6727,Ada apa pake topeng kurang nyaman kenapa mukan...,1208,https://youtu.be/2EYA2kxtSDM?si=zqUP4Tg1aopsrwHs,0,Perhatikan ini Sebelum Anda Membeli BYD ATTO 1...,comment,NaN,0
1,@NuhAbdullah-r2h,Wowww...,1208,https://youtu.be/2EYA2kxtSDM?si=zqUP4Tg1aopsrwHs,0,Perhatikan ini Sebelum Anda Membeli BYD ATTO 1...,comment,NaN,0
2,@ridwannaq,Klo dikasi pilihan mending brio atau ini? Untu...,1208,https://youtu.be/2EYA2kxtSDM?si=zqUP4Tg1aopsrwHs,0,Perhatikan ini Sebelum Anda Membeli BYD ATTO 1...,comment,NaN,0
3,@ogerep2877,"tolong yg mau beli ini, pikir masak2, tau send...",1208,https://youtu.be/2EYA2kxtSDM?si=zqUP4Tg1aopsrwHs,0,Perhatikan ini Sebelum Anda Membeli BYD ATTO 1...,comment,NaN,0
4,@Bos93-l1k,Kecepatan maksimal hanya 100kmð,1208,https://youtu.be/2EYA2kxtSDM?si=zqUP4Tg1aopsrwHs,0,Perhatikan ini Sebelum Anda Membeli BYD ATTO 1...,comment,NaN,0
...,...,...,...,...,...,...,...,...,...
95,@AlbertsBerts,Ditiap Chanel otomitive Toyota spam iklan ð...,1208,https://youtu.be/2EYA2kxtSDM?si=zqUP4Tg1aopsrwHs,0,Perhatikan ini Sebelum Anda Membeli BYD ATTO 1...,comment,NaN,0
96,@mancingperkoro1683,Coba bawa ke tanjakan sikarim Ama krakalan om ...,1208,https://youtu.be/2EYA2kxtSDM?si=zqUP4Tg1aopsrwHs,0,Perhatikan ini Sebelum Anda Membeli BYD ATTO 1...,comment,NaN,0
97,@cipluxgendeng9558,#HERAN_????,1208,https://youtu.be/2EYA2kxtSDM?si=zqUP4Tg1aopsrwHs,0,Perhatikan ini Sebelum Anda Membeli BYD ATTO 1...,comment,NaN,1
98,@koncomancing9746,Toyota raja diesel gak usah fokus mobil hybrid...,1208,https://youtu.be/2EYA2kxtSDM?si=zqUP4Tg1aopsrwHs,0,Perhatikan ini Sebelum Anda Membeli BYD ATTO 1...,comment,NaN,2


# **Merapihkan kolom**

In [763]:
df.columns = df.columns.str.strip()       # hapus spasi depan/belakang
df.columns = df.columns.str.replace('ï»¿"', '', regex=False)  # hapus BOM

In [764]:
df = df.drop(columns=['author"','commentsCount', 'pageUrl','replyCount','title','type','videoID','voteCount'])

df

,comment
0,Ada apa pake topeng kurang nyaman kenapa mukan...
1,Wowww...
2,Klo dikasi pilihan mending brio atau ini? Untu...
3,"tolong yg mau beli ini, pikir masak2, tau send..."
4,Kecepatan maksimal hanya 100kmð
...,...
95,Ditiap Chanel otomitive Toyota spam iklan ð...
96,Coba bawa ke tanjakan sikarim Ama krakalan om ...
97,#HERAN_????
98,Toyota raja diesel gak usah fokus mobil hybrid...


In [765]:
df.columns = df.columns.str.strip()       # hapus spasi depan/belakang
df.columns = df.columns.str.replace('ï»¿"', '', regex=False)  # hapus BOM

In [766]:
df.rename(columns={"comment" : "Comment"}, inplace=True)

df

,Comment
0,Ada apa pake topeng kurang nyaman kenapa mukan...
1,Wowww...
2,Klo dikasi pilihan mending brio atau ini? Untu...
3,"tolong yg mau beli ini, pikir masak2, tau send..."
4,Kecepatan maksimal hanya 100kmð
...,...
95,Ditiap Chanel otomitive Toyota spam iklan ð...
96,Coba bawa ke tanjakan sikarim Ama krakalan om ...
97,#HERAN_????
98,Toyota raja diesel gak usah fokus mobil hybrid...


# **Cleaning Text**

In [767]:
import re
import pandas as pd

# --- 1. Kamus kata tidak baku → kata baku ---
normalisasi_dict = {
    "gpp": "tidak apa apa",
    "bgt": "banget",
    "mantul": "mantap betul",
    "ga": "tidak",
    "nggak": "tidak",
    "aja": "saja",
    "klo": "kalau",
    "tdk": "tidak",
    "dr": "dari",
    "dgn": "dengan",
    "yg": "yang",
    "masak2": "matang matang"
}

# --- 2. Kamus brand/nama → format kapitalisasi benar ---
brand_dict = {
    "byd": "BYD",
    "toyota": "Toyota",
    "honda": "Honda",
    "wuling": "Wuling",
    "mitsubishi": "Mitsubishi",
    "hyundai": "Hyundai",
    "nissan": "Nissan",
    "suzuki": "Suzuki",
    "daihatsu": "Daihatsu",
    "tesla": "Tesla"
}

# --- Fungsi cleaning + normalisasi ---
def clean_text(text):
    # ubah ke lowercase
    text = str(text).lower()
    
    # hapus karakter non huruf/angka
    text = re.sub('[^a-z0-9 ]', ' ', text)
    
    # tokenisasi
    tokens = text.split()
    
    # normalisasi kata tidak baku
    tokens = [normalisasi_dict.get(word, word) for word in tokens]
    
    # normalisasi brand
    tokens = [brand_dict.get(word, word) for word in tokens]
    
    # gabungkan kembali
    return " ".join(tokens)

# --- Terapkan ke kolom comment ---
df['Cleaned_comment'] = df['Comment'].apply(clean_text)

# --- Lihat hasil ---
df_display = df[['Comment', 'Cleaned_comment']]
df_display   # tampil tabel langsung, tidak usah print()



,Comment,Cleaned_comment
0,Ada apa pake topeng kurang nyaman kenapa mukan...,ada apa pake topeng kurang nyaman kenapa mukan...
1,Wowww...,wowww
2,Klo dikasi pilihan mending brio atau ini? Untu...,kalau dikasi pilihan mending brio atau ini unt...
3,"tolong yg mau beli ini, pikir masak2, tau send...",tolong yang mau beli ini pikir matang matang t...
4,Kecepatan maksimal hanya 100kmð,kecepatan maksimal hanya 100km
...,...,...
95,Ditiap Chanel otomitive Toyota spam iklan ð...,ditiap chanel otomitive Toyota spam iklan pani...
96,Coba bawa ke tanjakan sikarim Ama krakalan om ...,coba bawa ke tanjakan sikarim ama krakalan om
97,#HERAN_????,heran
98,Toyota raja diesel gak usah fokus mobil hybrid...,Toyota raja diesel gak usah fokus mobil hybrid...


In [768]:
def count_punct(text):
    # kalau text kosong atau None, langsung return 0
    if not text or (len(text) - text.count(" ")) == 0:
        return 0
    
    count = sum([1 for char in text if char in string.punctuation])
    return round(count / (len(text) - text.count(" ")), 3) * 100

# hitung panjang teks tanpa spasi
df['Review_len'] = df['Cleaned_comment'].apply(lambda x: len(x) - x.count(" "))

# hitung persentase tanda baca
df['punct'] = df['Cleaned_comment'].apply(lambda x: count_punct(x))
df


,Comment,Cleaned_comment,Review_len,punct
0,Ada apa pake topeng kurang nyaman kenapa mukan...,ada apa pake topeng kurang nyaman kenapa mukan...,59,0.0
1,Wowww...,wowww,5,0.0
2,Klo dikasi pilihan mending brio atau ini? Untu...,kalau dikasi pilihan mending brio atau ini unt...,58,0.0
3,"tolong yg mau beli ini, pikir masak2, tau send...",tolong yang mau beli ini pikir matang matang t...,50,0.0
4,Kecepatan maksimal hanya 100kmð,kecepatan maksimal hanya 100km,27,0.0
...,...,...,...,...
95,Ditiap Chanel otomitive Toyota spam iklan ð...,ditiap chanel otomitive Toyota spam iklan pani...,51,0.0
96,Coba bawa ke tanjakan sikarim Ama krakalan om ...,coba bawa ke tanjakan sikarim ama krakalan om,38,0.0
97,#HERAN_????,heran,5,0.0
98,Toyota raja diesel gak usah fokus mobil hybrid...,Toyota raja diesel gak usah fokus mobil hybrid...,92,0.0


In [769]:
def tokenize_text(text):
    tokenized_text = text.split()
    return tokenized_text
df['tokens'] = df['Cleaned_comment'].apply(lambda x: tokenize_text(x))
df

,Comment,Cleaned_comment,Review_len,punct,tokens
0,Ada apa pake topeng kurang nyaman kenapa mukan...,ada apa pake topeng kurang nyaman kenapa mukan...,59,0.0,"[ada, apa, pake, topeng, kurang, nyaman, kenap..."
1,Wowww...,wowww,5,0.0,[wowww]
2,Klo dikasi pilihan mending brio atau ini? Untu...,kalau dikasi pilihan mending brio atau ini unt...,58,0.0,"[kalau, dikasi, pilihan, mending, brio, atau, ..."
3,"tolong yg mau beli ini, pikir masak2, tau send...",tolong yang mau beli ini pikir matang matang t...,50,0.0,"[tolong, yang, mau, beli, ini, pikir, matang, ..."
4,Kecepatan maksimal hanya 100kmð,kecepatan maksimal hanya 100km,27,0.0,"[kecepatan, maksimal, hanya, 100km]"
...,...,...,...,...,...
95,Ditiap Chanel otomitive Toyota spam iklan ð...,ditiap chanel otomitive Toyota spam iklan pani...,51,0.0,"[ditiap, chanel, otomitive, Toyota, spam, ikla..."
96,Coba bawa ke tanjakan sikarim Ama krakalan om ...,coba bawa ke tanjakan sikarim ama krakalan om,38,0.0,"[coba, bawa, ke, tanjakan, sikarim, ama, kraka..."
97,#HERAN_????,heran,5,0.0,[heran]
98,Toyota raja diesel gak usah fokus mobil hybrid...,Toyota raja diesel gak usah fokus mobil hybrid...,92,0.0,"[Toyota, raja, diesel, gak, usah, fokus, mobil..."


<h2>Lemmatization and Removing Stopwords</h2>

Lemmezation : mengubah kata ke bentuk dasar.

Stopwords Removal : Membuang kata - kata umum.

Tujuannya untuk menyatukan variasi kata dan menghilangkan noise.

In [770]:
nltk.download('stopwords')
from nltk.corpus import stopwords
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edlyn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [771]:
nltk.download('wordnet')
nltk.download('omw-1.4')
def lemmatize_text(token_list):
    return " ".join([lemmatizer.lemmatize(token) for token in token_list if not token in set(all_stopwords)])

lemmatizer = nltk.stem.WordNetLemmatizer()
df['lemmatized_review'] = df['tokens'].apply(lambda x: lemmatize_text(x))
df

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\edlyn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\edlyn\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Comment,Cleaned_comment,Review_len,punct,tokens,lemmatized_review
0,Ada apa pake topeng kurang nyaman kenapa mukan...,ada apa pake topeng kurang nyaman kenapa mukan...,59,0.0,"[ada, apa, pake, topeng, kurang, nyaman, kenap...",ada apa pake topeng kurang nyaman kenapa mukan...
1,Wowww...,wowww,5,0.0,[wowww],wowww
2,Klo dikasi pilihan mending brio atau ini? Untu...,kalau dikasi pilihan mending brio atau ini unt...,58,0.0,"[kalau, dikasi, pilihan, mending, brio, atau, ...",kalau dikasi pilihan mending brio atau ini unt...
3,"tolong yg mau beli ini, pikir masak2, tau send...",tolong yang mau beli ini pikir matang matang t...,50,0.0,"[tolong, yang, mau, beli, ini, pikir, matang, ...",tolong yang mau beli ini pikir matang matang t...
4,Kecepatan maksimal hanya 100kmð,kecepatan maksimal hanya 100km,27,0.0,"[kecepatan, maksimal, hanya, 100km]",kecepatan maksimal hanya 100km
...,...,...,...,...,...,...
95,Ditiap Chanel otomitive Toyota spam iklan ð...,ditiap chanel otomitive Toyota spam iklan pani...,51,0.0,"[ditiap, chanel, otomitive, Toyota, spam, ikla...",ditiap chanel otomitive Toyota spam iklan pani...
96,Coba bawa ke tanjakan sikarim Ama krakalan om ...,coba bawa ke tanjakan sikarim ama krakalan om,38,0.0,"[coba, bawa, ke, tanjakan, sikarim, ama, kraka...",coba bawa ke tanjakan sikarim ama krakalan om
97,#HERAN_????,heran,5,0.0,[heran],heran
98,Toyota raja diesel gak usah fokus mobil hybrid...,Toyota raja diesel gak usah fokus mobil hybrid...,92,0.0,"[Toyota, raja, diesel, gak, usah, fokus, mobil...",Toyota raja diesel gak usah fokus mobil hybrid...
